In [17]:
import pandas as pd
import json
import pytz

In [13]:
areas_df_dict = {
    'f_hertz': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_50Hertz_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'amprion': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_Amprion_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'tennet': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TenneT_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'transnet_bw': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TransnetBW_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    )
}

In [14]:
len(areas_df_dict['f_hertz'])

70176

In [23]:
with open('/home/aklei/Dropbox/Masterarbeit/Daten/SMARD/smard_api_amprion.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data['series'], columns=['timestamp', 'value_amprion'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df.index = df.index.tz_localize('UTC').tz_convert('Europe/Berlin')

print(df.tail())
print(len(df))
print(4*24*7)

                           value_amprion
timestamp                               
2023-01-01 22:45:00+01:00         401.00
2023-01-01 23:00:00+01:00         401.25
2023-01-01 23:15:00+01:00         400.25
2023-01-01 23:30:00+01:00         409.75
2023-01-01 23:45:00+01:00         405.75
672
672
